# Tests of the LSTM network to detect Slips behavioral letters

In [2]:
import sys
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from random import shuffle

#import lstm_dataset_functions
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Masking, Dropout
from keras.layers.recurrent import GRU
from keras.callbacks import ModelCheckpoint

In [59]:
dataset_file='datasets/all_datasets_raw'
with open(dataset_file, 'rb') as csvfile:
    df = pd.read_csv(csvfile, delimiter='|', names=[ "note", "label", "model_id", "state"], skipinitialspace=True)

In [60]:
# Clean the dataset
df.dropna(axis=0, how='any', inplace=True)

In [24]:
df.describe(include='all')

,note,label,model_id,state
count,7616,7555,7616,7574
unique,7555,7462,6459,2117
top,[ ],From-Botnet-TCP-Unknown--24 ...,[36m147.32.84.170-209.85.148.104-80-tcp[0m ...,2
freq,61,2,3,2203


In [25]:
def filter_by_string(df, col, string):
    ''' A function to filter the columns and rows'''
    return df[df[col].str.contains(string, regex=False) == True]

In [26]:
def split_data(data, split_pct=0.1):
        '''
        Splits data into training and testing.
        '''
        shuffle(data)
        return train_test_split(data, test_size=split_pct)

In [84]:
# Add a new column to the dataframe with the label. The label is 'Normal' for the normal data and 'Malcious' for the malware data
df.loc[df.label.str.contains('Normal'),'label'] = 'Normal'
df.loc[df.label.str.contains('Botnet'),'label'] = 'Malicious'
df.loc[df.label.str.contains('Malware'),'label'] = 'Malicious'

In [7]:
#normal_data = filter_by_string(df, 'label', 'Normal')['state'].values.tolist()

In [8]:
#botnet_data = filter_by_string(df, 'label', 'Botnet')['state'].values.tolist()

In [9]:
print('Amount of normal outtupless: {}. Amount of botnet outtuples: {}'.format(len(normal_data), len(botnet_data)))

Amount of normal outtupless: 1294. Amount of botnet outtuples: 6213


In [10]:
#y_data = [0 for i in range(len(normal_data))] + [1 for i in range(len(botnet_data))]

In [11]:
#data = zip(normal_data + botnet_data, y_data)

In [12]:
train_data, test_data = split_data(data, split_pct=0.2)

TypeError: object of type 'zip' has no len()

In [78]:
test_data

NameError: name 'test_data' is not defined